In [23]:
# Copyright 2025 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Getting Started with Google A2A (Agent-to-Agent) Communication

This notebook introduces you to Google's Agent-to-Agent (A2A) protocol, a standardized way for AI agents to communicate and collaborate.  

<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/a2aproject/a2a-samples/blob/main/notebooks/a2a_quickstart.ipynb">
      <img width="32px" src="https://www.gstatic.com/pantheon/images/bigquery/welcome_page/colab-logo.svg" alt="Google Colaboratory logo"><br> Open in Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https:%2F%2Fraw.githubusercontent.com%2Fa2aproject%2Fa2a-samples%2Fmain%2Fnotebooks%2Fa2a_quickstart.ipynb">
      <img width="32px" src="https://lh3.googleusercontent.com/JmcxdQi-qOpctIvWKgPtrzZdJJK-J3sWE1RsfjZNwshCFgE_9fULcNpuXYTilIR2hjwN" alt="Google Cloud Colab Enterprise logo"><br> Open in Colab Enterprise
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/a2aproject/a2a-samples/main/notebooks/a2a_quickstart.ipynb">
      <img src="https://www.gstatic.com/images/branding/gcpiconscolors/vertexai/v1/32px.svg" alt="Vertex AI logo"><br> Open in Vertex AI Workbench
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/bigquery/import?url=https://github.com/a2aproject/a2a-samples/blob/main/notebooks/a2a_quickstart.ipynb">
      <img src="https://www.gstatic.com/images/branding/gcpiconscolors/bigquery/v1/32px.svg" alt="BigQuery Studio logo"><br> Open in BigQuery Studio
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://github.com/a2aproject/a2a-samples/blob/main/notebooks/a2a_quickstart.ipynb">
      <img width="32px" src="https://www.svgrepo.com/download/217753/github.svg" alt="GitHub logo"><br> View on GitHub
    </a>
  </td>
</table>

## What You'll Build

A three-agent system that works together to analyze trending topics:
1. **Trending Topics Agent** - Searches the web for current trending topics
2. **Trend Analyzer Agent** - Performs deep analysis with quantitative data
3. **Host Agent** - Orchestrates the other agents to provide comprehensive insights

<img src="https://storage.googleapis.com/github-repo/a2a/a2a-diagram.png" alt="drawing" width="1000"/>

## Prerequisites

- Python 3.11+
- Google Cloud Project with Vertex AI enabled
- Basic understanding of async Python

## Other Resources

- [Google ADK Documentation](https://google.github.io/adk-docs/)
- [A2A Protocol Specification](https://github.com/google/a2a)
- [Vertex AI Documentation](https://cloud.google.com/vertex-ai)
- Codelabs:
  - [Google's Agent Stack in Action: ADK, A2A, MCP on Google Cloud](https://codelabs.developers.google.com/instavibe-adk-multi-agents/instructions)
  - [Getting Started with Agent-to-Agent (A2A) Protocol: Gemini on Cloud Run](https://codelabs.developers.google.com/intro-a2a-purchasing-concierge)
  - [Getting Started with MCP, ADK and A2A](https://codelabs.developers.google.com/codelabs/currency-agent)

#### Important!
A2A is a work in progress (WIP) thus, in the near future there might be changes that are different from what demonstrated in this code.

### Setup and Installation

First, let's install the required dependencies:

In [24]:
# Install required packages
%pip install --upgrade -q google-genai google-adk==1.9.0 a2a-sdk==0.3.0 python-dotenv aiohttp uvicorn requests mermaid-python nest-asyncio

Note: you may need to restart the kernel to use updated packages.


## 1. Introduction to A2A

### What is Agent-to-Agent (A2A) Communication?

A2A is a standardized protocol that enables AI agents to:
- **Discover** each other's capabilities
- **Communicate** using a common JSON-RPC based protocol
- **Collaborate** to solve complex tasks
- **Stream** responses for real-time interactions

### Environment Configuration

In [25]:
# Targeted workaround for google-adk==1.9.0 compatibility with a2a-sdk==0.3.0
# This cell shall be removed when google-adk releases the version next to >1.9.0
# (after https://github.com/google/adk-python/pull/2297)


import sys

from a2a.client import client as real_client_module
from a2a.client.card_resolver import A2ACardResolver


class PatchedClientModule:
    def __init__(self, real_module) -> None:
        for attr in dir(real_module):
            if not attr.startswith('_'):
                setattr(self, attr, getattr(real_module, attr))
        self.A2ACardResolver = A2ACardResolver


patched_module = PatchedClientModule(real_client_module)
sys.modules['a2a.client.client'] = patched_module  # type: ignore

In [26]:
import asyncio
import logging
import os
import sys
import threading
import time

from typing import Any

import httpx
import nest_asyncio
import uvicorn

from a2a.client import ClientConfig, ClientFactory, create_text_message_object
from a2a.server.apps import A2AStarletteApplication
from a2a.server.request_handlers import DefaultRequestHandler
from a2a.server.tasks import InMemoryTaskStore
from a2a.types import (
    AgentCapabilities,
    AgentCard,
    AgentSkill,
    TransportProtocol,
)
from a2a.utils.constants import AGENT_CARD_WELL_KNOWN_PATH
from dotenv import load_dotenv
from google.adk.a2a.executor.a2a_agent_executor import (
    A2aAgentExecutor,
    A2aAgentExecutorConfig,
)
from google.adk.agents import Agent, SequentialAgent
from google.adk.agents.remote_a2a_agent import RemoteA2aAgent
from google.adk.artifacts import InMemoryArtifactService
from google.adk.memory.in_memory_memory_service import InMemoryMemoryService
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search

In [27]:
# Set Google Cloud Configuration
os.environ['GOOGLE_GENAI_USE_VERTEXAI'] = 'FALSE'
os.environ['GOOGLE_CLOUD_PROJECT'] = (
    '[1001848614995]'  # @param {type: "string", placeholder: "[your-project-id]", isTemplate: true}
)
os.environ['GOOGLE_CLOUD_LOCATION'] = (
    'us-central1'  # Replace with your location
)

load_dotenv()

print('Environment variables configured:')
print(f'GOOGLE_GENAI_USE_VERTEXAI: {os.environ["GOOGLE_GENAI_USE_VERTEXAI"]}')
print(f'GOOGLE_CLOUD_PROJECT: {os.environ["GOOGLE_CLOUD_PROJECT"]}')
print(f'GOOGLE_CLOUD_LOCATION: {os.environ["GOOGLE_CLOUD_LOCATION"]}')

Environment variables configured:
GOOGLE_GENAI_USE_VERTEXAI: FALSE
GOOGLE_CLOUD_PROJECT: [1001848614995]
GOOGLE_CLOUD_LOCATION: us-central1


In [28]:
# Authenticate your notebook environment (Colab only)
if 'google.colab' in sys.modules:
    from google.colab import auth

    auth.authenticate_user(project_id=os.environ['GOOGLE_CLOUD_PROJECT'])

In [29]:
# Setup logging
logging.basicConfig(
    level=logging.ERROR,
    format='%(asctime)s - %(levelname)s - %(name)s - %(message)s',
)

## 2. Building Your A2A System

Let's build our three-agent system step by step. We'll create:

1. **Trending Topics Agent** - Finds current trending topics
2. **Trend Analyzer Agent** - Analyzes trends with quantitative data
3. **Host Agent** - Orchestrates the other agents (sequentially)

### Agent 1: Trending Topics Agent

This agent searches the web for trending topics and returns a list of current trends.

In [30]:
# Create the Trending Topics ADK Agent
trending_agent = Agent(
    model='gemini-2.5-pro',
    name='trending_topics_agent',
    instruction="""
    You are a social media trends analyst. Your job is to search the web for current trending topics,
    particularly from social platforms.

    When asked about trends:
    1. Search for "trending topics today" or similar queries
    2. Extract the top 3 trending topics
    3. Return them in a JSON format

    Focus on current, real-time trends from the last 24 hours.

    You MUST return your response in the following JSON format:
    {
        "trends": [
            {
                "topic": "Topic name",
                "description": "Brief description (1-2 sentences)",
                "reason": "Why it's trending"
            },
            {
                "topic": "Topic name",
                "description": "Brief description (1-2 sentences)",
                "reason": "Why it's trending"
            },
            {
                "topic": "Topic name",
                "description": "Brief description (1-2 sentences)",
                "reason": "Why it's trending"
            }
        ]
    }

    Only return the JSON object, no additional text.
    """,
    tools=[google_search],
)

print('Trending Topics Agent created successfully!')

Trending Topics Agent created successfully!


In [31]:
trending_agent_card = AgentCard(
    name='Trending Topics Agent',
    url='http://localhost:10020',
    description='Searches the web for current trending topics from social media',
    version='1.0',
    capabilities=AgentCapabilities(streaming=True),
    default_input_modes=['text/plain'],
    default_output_modes=['text/plain'],
    preferred_transport=TransportProtocol.jsonrpc,
    skills=[
        AgentSkill(
            id='find_trends',
            name='Find Trending Topics',
            description='Searches for current trending topics on social media',
            tags=['trends', 'social media', 'twitter', 'current events'],
            examples=[
                "What's trending today?",
                'Show me current Twitter trends',
                'What are people talking about on social media?',
            ],
        )
    ],
)

In [32]:
remote_trending_agent = RemoteA2aAgent(
    name='find_trends',
    description='Searches for current trending topics on social media',
    agent_card=f'http://localhost:10020{AGENT_CARD_WELL_KNOWN_PATH}',
)

C:\Users\Dawn\AppData\Local\Temp\ipykernel_23592\912843253.py:1: UserWarning: [EXPERIMENTAL] RemoteA2aAgent: This feature is experimental and may change or be removed in future versions without notice. It may introduce breaking changes at any time.
  remote_trending_agent = RemoteA2aAgent(


### Agent 2: Trend Analyzer Agent

This agent takes a specific trend and performs deep analysis with quantitative data.

In [33]:
# Create the Trend Analyzer ADK Agent
analyzer_agent = Agent(
    model='gemini-2.5-pro',
    name='trend_analyzer_agent',
    instruction="""
    You are a data analyst specializing in trend analysis. When given a trending topic,
    perform deep research to find quantitative data and insights.

    For each trend you analyze:
    1. Search for statistics, numbers, and metrics related to the trend
    2. Look for:
       - Engagement metrics (views, shares, mentions)
       - Growth rates and timeline
       - Geographic distribution
       - Related hashtags or keywords
    3. Provide concrete numbers and data points

    Keep it somehow concise

    Always prioritize quantitative information over qualitative descriptions.
    """,
    tools=[google_search],
)

print('Trend Analyzer Agent created successfully!')

Trend Analyzer Agent created successfully!


In [34]:
analyzer_agent_card = AgentCard(
    name='Trend Analyzer Agent',
    url='http://localhost:10021',
    description='Performs deep analysis of trends with quantitative data',
    version='1.0',
    capabilities=AgentCapabilities(streaming=True),
    default_input_modes=['text/plain'],
    default_output_modes=['text/plain'],
    preferred_transport=TransportProtocol.jsonrpc,
    skills=[
        AgentSkill(
            id='analyze_trend',
            name='Analyze Trend',
            description='Provides quantitative analysis of a specific trend',
            tags=['analysis', 'data', 'metrics', 'statistics'],
            examples=[
                'Analyze the #ClimateChange trend',
                'Get metrics for the Taylor Swift trend',
                'Provide data analysis for AI adoption trend',
            ],
        )
    ],
)

In [35]:
remote_analyzer_agent = RemoteA2aAgent(
    name='analyze_trend',
    description='Provides quantitative analysis of a specific trend',
    agent_card=f'http://localhost:10021{AGENT_CARD_WELL_KNOWN_PATH}',
)

C:\Users\Dawn\AppData\Local\Temp\ipykernel_23592\738464179.py:1: UserWarning: [EXPERIMENTAL] RemoteA2aAgent: This feature is experimental and may change or be removed in future versions without notice. It may introduce breaking changes at any time.
  remote_analyzer_agent = RemoteA2aAgent(


### Agent 3: Host Agent (Orchestrator)

The Host Agent coordinates between the other two agents to provide comprehensive trend analysis.

In [36]:
# Create the Host ADK Agent
host_agent = SequentialAgent(
    name='trend_analysis_host',
    sub_agents=[remote_trending_agent, remote_analyzer_agent],
)

In [37]:
host_agent_card = AgentCard(
    name='Trend Analysis Host',
    url='http://localhost:10022',
    description='Orchestrates, sequentially, trend discovery and analysis using specialized agents',
    version='1.0',
    capabilities=AgentCapabilities(streaming=True),
    default_input_modes=['text/plain'],
    default_output_modes=['application/json'],
    preferred_transport=TransportProtocol.jsonrpc,
    skills=[
        AgentSkill(
            id='comprehensive_trend_analysis',
            name='Comprehensive Trend Analysis',
            description='Finds trending topics and provides deep analysis of the most relevant one',
            tags=['trends', 'analysis', 'orchestration', 'insights'],
            examples=[
                'Analyze current trends',
                "What's trending and why is it important?",
                'Give me a comprehensive trend report',
            ],
        )
    ],
)

## 3. Running

Now let's put everything together. We'll create helper functions to start our agents and run the complete system.

### Starting the A2A Servers

Create function to run each agent as an A2A server:

In [38]:
def create_agent_a2a_server(agent, agent_card):
    """Create an A2A server for any ADK agent.

    Args:
        agent: The ADK agent instance
        agent_card: The ADK agent card

    Returns:
        A2AStarletteApplication instance
    """
    runner = Runner(
        app_name=agent.name,
        agent=agent,
        artifact_service=InMemoryArtifactService(),
        session_service=InMemorySessionService(),
        memory_service=InMemoryMemoryService(),
    )

    config = A2aAgentExecutorConfig()
    executor = A2aAgentExecutor(runner=runner, config=config)

    request_handler = DefaultRequestHandler(
        agent_executor=executor,
        task_store=InMemoryTaskStore(),
    )

    # Create A2A application
    return A2AStarletteApplication(
        agent_card=agent_card, http_handler=request_handler
    )

In [45]:
# Apply nest_asyncio
nest_asyncio.apply()

# Store server tasks
server_tasks: list[asyncio.Task] = []


async def run_agent_server(agent, agent_card, port) -> None:
    """Run a single agent server."""
    app = create_agent_a2a_server(agent, agent_card)

    config = uvicorn.Config(
        app.build(),
        host='127.0.0.1',
        port=port,
        log_level='warning',
        loop='none',  # Important: let uvicorn use the current loop
    )

    server = uvicorn.Server(config)
    await server.serve()


async def start_all_servers() -> None:
    """Start all servers in the same event loop."""
    # Create tasks for all servers
    tasks = [
        asyncio.create_task(
            run_agent_server(trending_agent, trending_agent_card, 10020)
        ),
        asyncio.create_task(
            run_agent_server(analyzer_agent, analyzer_agent_card, 10021)
        ),
        asyncio.create_task(
            run_agent_server(host_agent, host_agent_card, 10022)
        ),
    ]

    # Give servers time to start
    await asyncio.sleep(2)

    print('✅ All agent servers started!')
    print('   - Trending Agent: http://127.0.0.1:10020')
    print('   - Analyzer Agent: http://127.0.0.1:10021')
    print('   - Host Agent: http://127.0.0.1:10022')

    # Keep servers running
    try:
        await asyncio.gather(*tasks)
    except KeyboardInterrupt:
        print('Shutting down servers...')


# Run in a background thread


def run_servers_in_background() -> None:
    loop = asyncio.new_event_loop()
    asyncio.set_event_loop(loop)
    loop.run_until_complete(start_all_servers())


# Start the thread
server_thread = threading.Thread(target=run_servers_in_background, daemon=True)
server_thread.start()

# Wait for servers to be ready
time.sleep(3)

C:\Users\Dawn\AppData\Local\Temp\ipykernel_23592\2220589873.py:19: UserWarning: [EXPERIMENTAL] A2aAgentExecutorConfig: This feature is experimental and may change or be removed in future versions without notice. It may introduce breaking changes at any time.
  config = A2aAgentExecutorConfig()
C:\Users\Dawn\AppData\Local\Temp\ipykernel_23592\2220589873.py:20: UserWarning: [EXPERIMENTAL] A2aAgentExecutor: This feature is experimental and may change or be removed in future versions without notice. It may introduce breaking changes at any time.
  executor = A2aAgentExecutor(runner=runner, config=config)
ERROR:    [Errno 10048] error while attempting to bind on address ('127.0.0.1', 10020): [winerror 10048] only one usage of each socket address (protocol/network address/port) is normally permitted
ERROR:    [Errno 10048] error while attempting to bind on address ('127.0.0.1', 10021): [winerror 10048] only one usage of each socket address (protocol/network address/port) is normally permitte

## 4. Testing the System

### Call the A2A agents (the 2 remote agents, and the host agent that refers to the 2 remote agents as sub agents)

In [40]:
class A2ASimpleClient:
    """A2A Simple to call A2A servers."""

    def __init__(self, default_timeout: float = 240.0):
        self._agent_info_cache: dict[
            str, dict[str, Any] | None
        ] = {}  # Cache for agent metadata
        self.default_timeout = default_timeout

    async def create_task(self, agent_url: str, message: str) -> str:
        """Send a message following the official A2A SDK pattern."""
        # Configure httpx client with timeout
        timeout_config = httpx.Timeout(
            timeout=self.default_timeout,
            connect=10.0,
            read=self.default_timeout,
            write=10.0,
            pool=5.0,
        )

        async with httpx.AsyncClient(timeout=timeout_config) as httpx_client:
            # Check if we have cached agent card data
            if (
                agent_url in self._agent_info_cache
                and self._agent_info_cache[agent_url] is not None
            ):
                agent_card_data = self._agent_info_cache[agent_url]
            else:
                # Fetch the agent card
                agent_card_response = await httpx_client.get(
                    f'{agent_url}{AGENT_CARD_WELL_KNOWN_PATH}'
                )
                agent_card_data = self._agent_info_cache[agent_url] = (
                    agent_card_response.json()
                )

            # Create AgentCard from data
            agent_card = AgentCard(**agent_card_data)

            # Create A2A client with the agent card
            config = ClientConfig(
                httpx_client=httpx_client,
                supported_transports=[
                    TransportProtocol.jsonrpc,
                    TransportProtocol.http_json,
                ],
                use_client_preference=True,
            )

            factory = ClientFactory(config)
            client = factory.create(agent_card)

            # Create the message object
            message_obj = create_text_message_object(content=message)

            # Send the message and collect responses
            responses = []
            async for response in client.send_message(message_obj):
                responses.append(response)

            # The response is a tuple - get the first element (Task object)
            if (
                responses
                and isinstance(responses[0], tuple)
                and len(responses[0]) > 0
            ):
                task = responses[0][0]  # First element of the tuple

                # Extract text: task.artifacts[0].parts[0].root.text
                try:
                    return task.artifacts[0].parts[0].root.text
                except (AttributeError, IndexError):
                    return str(task)

            return 'No response received'

In [41]:
a2a_client = A2ASimpleClient()

In [42]:
async def test_trending_topics() -> None:
    """Test trending topics agent."""
    trending_topics = await a2a_client.create_task(
        'http://localhost:10020', "What's trending today?"
    )
    print(trending_topics)


# Run the async function
asyncio.run(test_trending_topics())

```json
{
    "trends": [
        {
            "topic": "Karur Stampede",
            "description": "A deadly stampede occurred at a political rally for actor-politician Vijay's Tamilaga Vettri Kazhagam (TVK) party in Karur, Tamil Nadu, India. The incident resulted in at least 40 deaths and numerous injuries.",
            "reason": "The tragic loss of life at a high-profile political event has led to widespread news coverage and social media discussion. Conversations are focused on the causes of the stampede, accountability, and the compensation announced for victims' families."
        },
        {
            "topic": "Asia Cup 2025 Final",
            "description": "India and Pakistan are facing off in the historic final of the Asia Cup 2025 cricket tournament for the first time. The highly anticipated match is being held in Dubai.",
            "reason": "The cricket rivalry between India and Pakistan is one of the most intense in the world, generating massive viewership and so

In [43]:
async def test_analysis() -> None:
    """Test analysis agent."""
    analysis = await a2a_client.create_task(
        'http://localhost:10021', 'Analyze the trend AI in Social Media'
    )
    print(analysis)


# Run the async function
asyncio.run(test_analysis())

2025-09-28 08:17:41,556 - ERROR - asyncio - Task was destroyed but it is pending!
task: <Task pending name='Task-45' coro=<run_agent_server() running at C:\Users\Dawn\AppData\Local\Temp\ipykernel_23592\1830070841.py:21> wait_for=<Future finished result=True>>
2025-09-28 08:17:41,557 - ERROR - asyncio - Task was destroyed but it is pending!
task: <Task pending name='Task-44' coro=<run_agent_server() running at C:\Users\Dawn\AppData\Local\Temp\ipykernel_23592\1830070841.py:21> wait_for=<Future finished result=True>>
2025-09-28 08:17:41,558 - ERROR - asyncio - Task exception was never retrieved
future: <Task finished name='Task-43' coro=<run_agent_server() done, defined at C:\Users\Dawn\AppData\Local\Temp\ipykernel_23592\1830070841.py:8> exception=SystemExit(1)>
Traceback (most recent call last):
  File "C:\Users\Dawn\AppData\Roaming\Python\Python313\site-packages\uvicorn\server.py", line 164, in startup
    server = await loop.create_server(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
    ...

## The Meteoric Rise of AI in Social Media: A Data-Driven Analysis

Artificial intelligence is no longer a futuristic concept in the social media landscape; it's a dominant force reshaping how users interact with platforms and how businesses engage with their audiences. A deep dive into the numbers reveals a clear and quantifiable trend of rapid AI integration and significant impact on engagement, content creation, and market growth.

### Market Size & Growth: A Multi-Billion Dollar Trajectory

The market for AI in social media is experiencing explosive growth, with various reports projecting a significant upward trend. The global AI in social media market was valued at approximately $2.2 billion in 2024 and is projected to reach $10.3 billion by 2029, growing at a CAGR of 36.2%. Another forecast estimates the market will surge to $12 billion by 2031, a substantial leap from $2.1 billion in 2021. Some projections are even more ambitious, suggesting the market could reach $54.07 billion

In [44]:
async def test_host_analysis() -> None:
    """Test host analysis agent."""
    host_analysis = await a2a_client.create_task(
        'http://localhost:10022',
        'Find the most relevant trends in the web today, choose randomly one of the top '
        'trends, and give me a complete analysis of it with quantitative data',
    )
    print(host_analysis)


# Run the async function
asyncio.run(test_host_analysis())

2025-09-28 08:18:14,710 - ERROR - asyncio - Unclosed connector
connections: ['deque([(<aiohttp.client_proto.ResponseHandler object at 0x0000019234568290>, 118903.93969)])']
connector: <aiohttp.connector.TCPConnector object at 0x0000019244FEB110>


### Asia Cup Final: A Quantitative Analysis of a High-Stakes Rivalry

The 2025 Asia Cup final, featuring the first-ever championship clash between cricket arch-rivals India and Pakistan in the tournament's 41-year history, has ignited a massive global trend. This analysis delves into the quantitative data driving the conversation, from record-breaking viewership and lucrative advertising to explosive social media engagement.

**Viewership and Commercial Impact: A Billion-Eyeball Spectacle**

While official viewership figures for the 2025 final are still pending, historical data from similar high-stakes encounters provides a clear indication of the massive audience.

*   **Past Viewership Benchmarks:** Previous India-Pakistan matches have consistently drawn colossal audiences. The 2019 World Cup clash attracted 273 million unique TV viewers and over 50 million digital-only viewers globally. The 2022 T20 World Cup match saw a peak of 18 million concurrent viewers on a single streaming pl

## Summary

Congratulations! You've successfully built a multi-agent system using Google's A2A protocol. Here's what you've learned:

1. **A2A Protocol Basics**: How agents discover and communicate with each other
2. **ADK Integration**: Creating ADK agents and wrapping them for A2A
3. **Agent Orchestration**: Building a Host Agent that coordinates multiple agents
4. **Practical Implementation**: Running and testing a complete multi-agent system

### Next Steps

- **Deploy Your Agents**: Deploy agents to Cloud Run or other platforms
- **Add Authentication**: Implement security for production use
- **Create More Agents**: Build agents for your specific use cases, even using other frameworks
- **Advanced Patterns**: Explore agent chains, parallel execution, and more
- **Callbacks**: Add in the Google ADK agents the before and after callbacks of the agent, model and tool, to increase observability

Happy agent building! 🚀

# Appendix

### Why Use Google A2A (Agent-to-Agent) Protocol

Google's Agent-to-Agent (A2A) protocol is a standardized communication framework that enables AI agents to discover, communicate, and collaborate with each other using a common JSON-RPC based protocol.  
It provides a uniform way for agents to interact, regardless of their underlying implementation.  

#### 1. Standardized Communication Protocol

- A2A provides a consistent, JSON-RPC based protocol that any agent can implement
- Agents can communicate without needing to know each other's internal implementation details
- The protocol supports streaming responses for real-time interactions

#### 2. Agent Discovery and Metadata

- Agents expose their capabilities through standardized metadata (AgentCard)
- Each agent publishes its skills, input/output modes, and capabilities
- Host agents can dynamically discover what other agents can do through the `.well-known/agent-card.json` endpoint

#### 3. Orchestration and Composition

- Enables building complex multi-agent systems where a host agent can orchestrate multiple specialized agents
- Supports sequential and parallel task execution patterns
- Allows for sophisticated agent collaboration workflows

#### 4. Platform Independence

- A2A servers can wrap agents from different frameworks (not just ADK)
- Agents can be deployed as independent services on different infrastructure
- Promotes loose coupling between agents

### Differences: Using ADK Agents Directly vs. Through A2A

#### Using ADK Agents Directly

```python
# Conceptual Example: Defining Hierarchy
from google.adk.agents import LlmAgent, BaseAgent

# Define individual agents
greeter = LlmAgent(name="Greeter", model="gemini-2.5-pro")
task_doer = BaseAgent(name="TaskExecutor") # Custom non-LLM agent

# Create parent agent and assign children via sub_agents
coordinator = LlmAgent(
    name="Coordinator",
    model="gemini-2.5-pro",
    description="I coordinate greetings and tasks.",
    sub_agents=[ # Assign sub_agents here
        greeter,
        task_doer
    ]
)
```

__Use Direct ADK for Multi-Agents System When:__

- All agents are tightly related and always used together
- Google ADK is the framework choice, and simplicity is prioritized
- Performance of in-process communication is critical
- You don't need distributed deployment
- No built-in service discovery is needed

#### Using ADK Agents Through A2A

__Use A2A for Multi-Agents System When:__

- Building complex multi-agent systems
- Agents need to be developed, deployed, and scaled independently
- You want to integrate agents from different teams or frameworks
- You need dynamic agent discovery and composition
- Building a platform where agents can be added/removed dynamically
- You want to enable third-party agent integration